In [ ]:
# Create the directory structure
import os

dirs = [
    'Trust_me_Im_wrong',
    'Trust_me_Im_wrong/semantic_uncertainty',
    'Trust_me_Im_wrong/semantic_uncertainty/uncertainty',

    'Trust_me_Im_wrong/semantic_uncertainty/uncertainty/models',
    'Trust_me_Im_wrong/semantic_uncertainty/uncertainty/uncertainty_measures',

]

for dir_path in dirs:
    os.makedirs(dir_path, exist_ok=True)

print("Directory structure created!")

Directory structure created!


In [ ]:

import os

dirs = [
    'Trust_me_Im_wrong/analysis',
    'Trust_me_Im_wrong//visualization',
     'Trust_me_Im_wrong/reporting',
     'Trust_me_Im_wrong/reports',
     'Trust_me_Im_wrong/visualizations'
]
for dir_path in dirs:
    os.makedirs(dir_path, exist_ok=True)

In [ ]:
# Create __init__.py in each directory to mark as package
for dir_path in dirs:
    init_path = os.path.join(dir_path, '__init__.py')
    if not os.path.exists(init_path):
        with open(init_path, 'w', encoding='utf-8') as f:
            f.write('# Init file for package\n')
        print(f'Created __init__.py in {dir_path}')
    else:
        print(f'__init__.py already exists in {dir_path}')

__init__.py already exists in Trust_me_Im_wrong/analysis
__init__.py already exists in Trust_me_Im_wrong//visualization
__init__.py already exists in Trust_me_Im_wrong/reporting
__init__.py already exists in Trust_me_Im_wrong/reports
__init__.py already exists in Trust_me_Im_wrong/visualizations


In [ ]:
%%writefile /content/Trust_me_Im_wrong/semantic_uncertainty/uncertainty/models/base_model.py
from abc import ABC, abstractmethod
from typing import List, Text, Optional, Tuple

# Full stop sequences for post-processing (all languages)
FULL_STOP_SEQUENCES = [
    '\n',
]

# API-compatible stop sequences (max 4 for OpenAI/DeepSeek API)
API_STOP_SEQUENCES = [
    '\n',       # Double newline - most common separator
    '.',        # Period - ends sentences
]

# Keep original for backward compatibility
STOP_SEQUENCES = FULL_STOP_SEQUENCES


class BaseModel(ABC):
    """Base model class with enhanced stop sequence handling (Malayalam Tailored)."""

    # Class variables
    stop_sequences: List[Text] = FULL_STOP_SEQUENCES
    api_stop_sequences: List[Text] = API_STOP_SEQUENCES

    def __init__(self):
        """Initialize with both API and full stop sequences."""
        self.stop_sequences = FULL_STOP_SEQUENCES
        self.api_stop_sequences = API_STOP_SEQUENCES

    @abstractmethod
    def predict(self, input_data: str, temperature: float):
        """
        Generate a response from the model given input_data and temperature.
        """
        pass

    @abstractmethod
    def get_p_true(self, input_data: str):
        """
        Compute probability that the answer to input_data is 'True'.
        """
        pass

    @staticmethod
    def post_process_with_stops(
        text: str,
        stop_sequences: Optional[List[str]] = None,
        preserve_stop: bool = False
    ) -> str:
        """
        Post-process text by truncating at the first occurrence of any stop sequence.

        Args:
            text: Input text to process
            stop_sequences: List of stop sequences (uses FULL_STOP_SEQUENCES if None)
            preserve_stop: If True, include the stop sequence in output

        Returns:
            Truncated text
        """
        if not text:
            return text

        if stop_sequences is None:
            stop_sequences = FULL_STOP_SEQUENCES

        # Find the earliest occurrence of any stop sequence
        earliest_pos = len(text)
        earliest_stop = None

        for stop in stop_sequences:
            pos = text.find(stop)
            if pos != -1 and pos < earliest_pos:
                earliest_pos = pos
                earliest_stop = stop

        # Truncate at the earliest stop sequence
        if earliest_pos < len(text):
            if preserve_stop and earliest_stop:
                return text[:earliest_pos + len(earliest_stop)]
            else:
                return text[:earliest_pos]

        return text

    @staticmethod
    def clean_for_comparison(text: str) -> str:
        """
        Aggressively clean text for accurate comparison and hallucination detection.
        Tailored for Malayalam language processing.

        Args:
            text: Text to clean

        Returns:
            Cleaned text
        """
        if not text:
            return ""

        # Convert to lowercase (works for standard chars, Malayalam chars remain largely unaffected but safe)
        text = text.lower()

        # Remove common Malayalam articles, conjunctions, and fillers (Stop Words)
        # Based on common linguistic patterns in Malayalam
        remove_words = [
            'ഒരു', 'ഈ', 'ആ', 'അത', 'ഇത', 'എന്', 'എന്ന', 'ആണ്', 'ഉം',
            'അല്ല', 'ഉണ്ട്', 'ഇല്ല', 'മാത്രം', 'വരെ', 'മുതൽ', 'നിന്ന്',
            'കൂടെ', 'പറ്റി', 'കൊണ്ട്', 'വേണ്ടി', 'തമ്മിൽ', 'പോലെ',
            'മറ്റും', 'എങ്കിൽ', 'അതെ', 'അതാ', 'ഇതാ', 'വളരെ', 'ഏറ്റവും'
        ]

        for word in remove_words:
            # Replace word with surrounding spaces to avoid breaking inside other words
            text = text.replace(f" {word} ", " ")

            # Check boundaries if the word is at start or end
            if text.startswith(f"{word} "):
                text = text[len(word)+1:]
            if text.endswith(f" {word}"):
                text = text[:-len(word)-1]

        # Remove all punctuation and special characters
        # Includes standard punctuation and common symbols
        special_chars = [
             '!', '?', ';', ',', '.', ':', '"', "'", "-", "_", "(", ")", "[", "]", "{", "}",
             '|', '/', '\\', '@', '#', '$', '%', '^', '&', '*'
        ]

        for char in special_chars:
            text = text.replace(char, ' ')

        # Normalize whitespace (collapse multiple spaces into one)
        text = ' '.join(text.split())

        return text.strip()

    @staticmethod
    def get_api_compatible_stops() -> List[str]:
        """
        Get API-compatible stop sequences (max 4).

        Returns:
            List of up to 4 stop sequences for API calls
        """
        return API_STOP_SEQUENCES[:4]

    @staticmethod
    def get_all_stops() -> List[str]:
        """
        Get all stop sequences for post-processing.

        Returns:
            Complete list of stop sequences
        """
        return FULL_STOP_SEQUENCES

Overwriting /content/Trust_me_Im_wrong/semantic_uncertainty/uncertainty/models/base_model.py


In [ ]:
%%writefile /content/Trust_me_Im_wrong/calc_semantic_entropy_api.py
"""
Semantic Entropy Calculation for API Models (DeepSeek V3 Strict - Malayalam)
Uses DeepSeek API with logprobs and MULTILINGUAL sentence transformers for clustering.
Strictly uses DeepSeek V3 API and Hugging Face AutoTokenizer.
Adapted for Malayalam datasets.
"""

import json
import logging
import random
from collections import defaultdict
import numpy as np
from openai import OpenAI
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
from google.colab import userdata
import sys
sys.path.append('/content/Trust_me_Im_wrong')

from semantic_uncertainty.uncertainty.models.base_model import (
    API_STOP_SEQUENCES, FULL_STOP_SEQUENCES, BaseModel
)


class SemanticEntropyAPI:
    def __init__(self, model_name="deepseek-chat", dataset_path="datasets/",
                 entailment_model="sentence_transformer", max_new_tokens=10):
        """
        Initialize semantic entropy calculator strictly for DeepSeek API (V3).
        Configured for Malayalam language processing.
        """
        random.seed(0)
        self.model_name = model_name  # "deepseek-chat" points to V3
        self.max_new_tokens = max_new_tokens

        # 1. Initialize API client for DeepSeek (Strict: No Fallback)
        try:
            api_key = userdata.get('deepseek')
            if not api_key:
                raise ValueError("DeepSeek API key not found in secrets.")

            self.client = OpenAI(
                api_key=api_key,
                base_url="https://api.deepseek.com"
            )
        except Exception as e:
            raise ValueError(f"Failed to initialize DeepSeek API client: {e}")

        # 2. Initialize Tokenizer (Strict: Hugging Face AutoTokenizer Only)
        try:
            hf_token = userdata.get('hftoken')
            if not hf_token:
                 print("Warning: 'hftoken' secret not found. Tokenizer load might fail if model is gated.")

            # Use DeepSeek-V3 tokenizer
            tokenizer_path = "deepseek-ai/DeepSeek-V3"
            print(f"Loading DeepSeek tokenizer from {tokenizer_path}...")

            self.tokenizer = AutoTokenizer.from_pretrained(
                tokenizer_path,
                token=hf_token,
                trust_remote_code=True
            )
        except Exception as e:
             raise RuntimeError(f"CRITICAL: Failed to load DeepSeek tokenizer via AutoTokenizer. \nError: {e}\nEnsure 'hftoken' is set correctly.")

        # Initialize sentence transformer for semantic similarity
        # CRITICAL: Using multilingual model for Malayalam support
        print("Loading multilingual embedding model for Malayalam semantic clustering...")
        self.embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

        # Clustering threshold (Malayalam is agglutinative, 0.5 is a safe start)
        self.clustering_threshold = 0.5

        # Setup stop sequences
        self.api_stops = API_STOP_SEQUENCES[:4]
        self.full_stops = FULL_STOP_SEQUENCES

        print(f"Initialized SemanticEntropyAPI with {model_name} (DeepSeek Strict - Malayalam)")
        print(f"Clustering threshold: {self.clustering_threshold}")

    def generate_answers(
        self,
        prompt: str,
        answer: str,
        num_generations: int = 11,
        temperature: float = 1.0,
        compute_acc: bool = False
    ):
        """
        Generate multiple answers from API model for entropy calculation.
        """
        generations = {}
        generations[prompt] = {"question": prompt}
        full_responses = []
        all_generation_texts = []  # Store all generated texts for output

        # System message for consistency (Malayalam)
        # Translates to: "Give only direct, short answers. Do not build sentences, do not give explanations."
        system_message = "നേരിട്ടുള്ളതും ഹ്രസ്വവുമായ ഉത്തരങ്ങൾ മാത്രം നൽകുക. വാചകങ്ങൾ നിർമ്മിക്കരുത്, വിശദീകരണം നൽകരുത്."

        print(f"Generating {num_generations} responses...")

        for i in range(num_generations):
            # First generation at low temperature
            temp = 0.1 if i == 0 else temperature

            # Get response with logprobs
            response_data = self._get_api_response(prompt, temp, system_message)

            # Store the text for output
            all_generation_texts.append(response_data["text"])

            if i == 0:
                # Store most likely answer
                most_likely_answer_dict = {
                    "response": response_data["text"],
                    "token_log_likelihoods": response_data["token_logprobs"],
                    "embedding": None,
                    "accuracy": self._check_accuracy(response_data["text"], answer) if compute_acc else 0.0,
                    "total_logprob": response_data["total_logprob"]
                }
                generations[prompt]["most_likely_answer"] = most_likely_answer_dict
            else:
                # Store high temperature responses
                full_responses.append((
                    response_data["text"],
                    response_data["token_logprobs"],
                    None,
                    self._check_accuracy(response_data["text"], answer) if compute_acc else 0.0,
                    response_data["total_logprob"]  # Add total logprob
                ))

        generations[prompt]["responses"] = full_responses
        generations[prompt]["reference"] = answer
        generations[prompt]["all_generation_texts"] = all_generation_texts  # Add all texts

        print(f"Generated texts: {all_generation_texts[:3]}...")  # Show first 3

        return {
            "accuracies": [most_likely_answer_dict["accuracy"]] if generations[prompt].get("most_likely_answer") else [],
            "generations": generations,
            "question": prompt,
            "reference": answer,
            "all_generation_texts": all_generation_texts
        }

    def _get_api_response(self, prompt: str, temperature: float, system_message: str = None):
        """Get response from DeepSeek API with logprobs."""
        try:
            messages = []
            if system_message:
                messages.append({"role": "system", "content": system_message})
            messages.append({"role": "user", "content": prompt})

            # DeepSeek API call
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                temperature=temperature,
                max_tokens=self.max_new_tokens,
                logprobs=True,
                top_logprobs=20,
                stop=self.api_stops
            )

            choice = response.choices[0]
            raw_text = choice.message.content or ""

            # Post-process with full stop sequences
            processed_text = BaseModel.post_process_with_stops(
                raw_text,
                self.full_stops,
                preserve_stop=False
            )

            # Extract logprobs
            total_logprob = 0
            token_logprobs = []

            if hasattr(choice, 'logprobs') and choice.logprobs and choice.logprobs.content:
                for logprob_data in choice.logprobs.content:
                    token_logprobs.append(logprob_data.logprob)
                    total_logprob += logprob_data.logprob

            return {
                "text": processed_text,
                "raw_text": raw_text,
                "token_logprobs": token_logprobs,
                "total_logprob": total_logprob  # Sum of all token logprobs
            }

        except Exception as e:
            print(f"API error: {e}")
            return {
                "text": "",
                "raw_text": "",
                "token_logprobs": [],
                "total_logprob": 0
            }

    def _check_accuracy(self, generated: str, target: str) -> float:
        """Check if generated text matches target (Internal basic check)."""
        gen_clean = BaseModel.clean_for_comparison(generated)
        target_clean = BaseModel.clean_for_comparison(target)

        if not gen_clean or not target_clean:
            return 0.0

        return 1.0 if (target_clean in gen_clean or gen_clean in target_clean) else 0.0

    def compute_uncertainty_measures(
        self,
        model_generations,
        compute_predictive_entropy=True,
        strict_entailment=False
    ):
        """
        Compute semantic entropy and other uncertainty measures using clustering.
        """
        result_dict = {"semantic_ids": [], "debug_info": {}}
        entropies = defaultdict(list)

        for tid in model_generations:
            example = model_generations[tid]
            full_responses = example.get("responses", [])

            # Include the most likely answer in calculations
            if "most_likely_answer" in example:
                most_likely = example["most_likely_answer"]
                full_responses.insert(0, (
                    most_likely["response"],
                    most_likely["token_log_likelihoods"],
                    None,
                    most_likely.get("accuracy", 0),
                    most_likely.get("total_logprob", sum(most_likely["token_log_likelihoods"]))
                ))

            if not full_responses:
                continue

            print(f"\nProcessing {len(full_responses)} total generations")

            # Clean responses before clustering
            raw_responses = [r[0] for r in full_responses]
            responses = [BaseModel.post_process_with_stops(r, self.full_stops) for r in raw_responses]

            if compute_predictive_entropy:
                # Get TOTAL log likelihoods (not per token average)
                total_log_liks = []
                for r in full_responses:
                    if len(r) >= 5:  # Has total_logprob
                        total_log_liks.append(r[4])
                    elif r[1]:  # Fallback to sum of token logprobs
                        total_log_liks.append(sum(r[1]))
                    else:
                        total_log_liks.append(-10)  # Default low value

                print(f"Total log likelihoods: {total_log_liks[:3]}...")

                # Compute semantic IDs using clustering
                semantic_ids = self._get_semantic_ids_clustering(responses)
                result_dict["semantic_ids"].append(semantic_ids)

                print(f"Semantic IDs: {semantic_ids}")
                unique_clusters = len(set(sid for sid in semantic_ids if sid >= 0))
                print(f"Number of semantic clusters: {unique_clusters}")

                # Store debug info
                result_dict["debug_info"]["num_generations"] = len(full_responses)
                result_dict["debug_info"]["num_clusters"] = unique_clusters
                result_dict["debug_info"]["responses_sample"] = responses[:3]

                # Compute cluster assignment entropy
                cluster_entropy = self._cluster_assignment_entropy(semantic_ids)
                entropies["cluster_assignment_entropy"].append(cluster_entropy)
                print(f"Cluster assignment entropy: {cluster_entropy:.4f}")

                # Regular entropy - using total log likelihoods
                regular_entropy = self._predictive_entropy_corrected(total_log_liks)
                entropies["regular_entropy"].append(regular_entropy)
                print(f"Regular entropy: {regular_entropy:.4f}")

                # Semantic entropy - aggregate by cluster then compute entropy
                log_likelihood_per_semantic_id = self._logsumexp_by_id(
                    semantic_ids, total_log_liks
                )
                semantic_entropy = self._semantic_entropy_corrected(log_likelihood_per_semantic_id)
                entropies["semantic_entropy"].append(semantic_entropy)
                print(f"Semantic entropy: {semantic_entropy:.4f}")

        # Compute averages
        avg_entropies = {k: np.mean(v) if v else 0 for k, v in entropies.items()}

        return avg_entropies, result_dict

    def _get_semantic_ids_clustering(self, responses: list) -> list:
        """Cluster responses using sentence embeddings."""
        if not responses or len(responses) <= 1:
            return [0] * len(responses)

        valid_responses = [r for r in responses if r and r.strip()]
        if not valid_responses:
            return [0] * len(responses)

        # Show what we're clustering
        print(f"Clustering {len(valid_responses)} responses")
        print(f"Sample responses: {valid_responses[:2]}")

        try:
            embeddings = self.embedding_model.encode(valid_responses)
        except Exception as e:
            print(f"Embedding error: {e}")
            return [0] * len(responses)

        if len(valid_responses) == 1:
            return [0] * len(responses)

        clustering = AgglomerativeClustering(
            n_clusters=None,
            distance_threshold=self.clustering_threshold,
            linkage='average'
        )

        try:
            semantic_ids = clustering.fit_predict(embeddings)
            full_ids = []
            valid_idx = 0
            for r in responses:
                if r and r.strip():
                    full_ids.append(semantic_ids[valid_idx])
                    valid_idx += 1
                else:
                    full_ids.append(-1)
            return full_ids
        except Exception as e:
            print(f"Clustering error: {e}")
            return [0] * len(responses)

    def _cluster_assignment_entropy(self, semantic_ids: list) -> float:
        """Calculate entropy of cluster assignments."""
        if not semantic_ids:
            return 0.0

        valid_ids = [sid for sid in semantic_ids if sid >= 0]
        if not valid_ids:
            return 0.0

        unique, counts = np.unique(valid_ids, return_counts=True)
        probs = counts / len(valid_ids)
        entropy = -np.sum(probs * np.log(probs + 1e-10))
        return float(entropy)

    def _predictive_entropy_corrected(self, total_log_liks: list) -> float:
        """
        CORRECTED: Calculate predictive entropy from log likelihoods.
        H = -E[log p(y|x)] ≈ -mean(log_likelihood)
        """
        if not total_log_liks:
            return 0.0

        # Average negative log likelihood
        avg_neg_log_lik = -np.mean(total_log_liks)
        return float(avg_neg_log_lik)

    def _semantic_entropy_corrected(self, log_likelihood_per_semantic_id: list) -> float:
        """
        CORRECTED: Calculate semantic entropy from aggregated log likelihoods.
        """
        if not log_likelihood_per_semantic_id:
            return 0.0

        # Convert log likelihoods to probabilities
        max_log_lik = max(log_likelihood_per_semantic_id)
        log_probs_normalized = [ll - max_log_lik for ll in log_likelihood_per_semantic_id]
        probs = np.exp(log_probs_normalized)
        probs = probs / (np.sum(probs) + 1e-10)

        # Calculate entropy
        entropy = -np.sum(probs * np.log(probs + 1e-10))
        return float(entropy)

    def _logsumexp_by_id(self, semantic_ids: list, total_log_liks: list) -> list:
        """
        Aggregate log probabilities by semantic ID using logsumexp.
        """
        if not semantic_ids or not total_log_liks:
            return []

        valid_pairs = [(sid, ll) for sid, ll in zip(semantic_ids, total_log_liks)
                       if sid >= 0 and ll is not None]
        if not valid_pairs:
            return []

        # Group by semantic ID
        id_to_log_liks = defaultdict(list)
        for sid, ll in valid_pairs:
            id_to_log_liks[sid].append(ll)

        # Aggregate using logsumexp for each semantic ID
        log_likelihood_per_id = []
        for sid in sorted(id_to_log_liks.keys()):
            lls = id_to_log_liks[sid]
            if lls:
                # Logsumexp: log(sum(exp(x_i)))
                max_ll = max(lls)
                sum_exp = sum(np.exp(ll - max_ll) for ll in lls)
                aggregated_ll = max_ll + np.log(sum_exp)
                log_likelihood_per_id.append(aggregated_ll)

        return log_likelihood_per_id

    def calc_semantic_entropy_per_example(
        self,
        prompt: str,
        answer: str,
        temp: float = 1.0,
        num_generations: int = 11
    ):
        """Calculate semantic entropy for a single example with all generations."""
        print(f"\nCalculating semantic entropy with {num_generations} generations (temp={temp:.1f})...")

        results = self.generate_answers(
            prompt=prompt,
            answer=answer,
            num_generations=num_generations,
            temperature=temp,
            compute_acc=True
        )

        avg_entropies, extra_info = self.compute_uncertainty_measures(
            results["generations"],
            compute_predictive_entropy=True,
            strict_entailment=False
        )

        # Add all generation texts to the output
        avg_entropies["all_generations"] = results.get("all_generation_texts", [])
        avg_entropies["debug_info"] = extra_info.get("debug_info", {})

        return avg_entropies, results["generations"]

Overwriting /content/Trust_me_Im_wrong/calc_semantic_entropy_api.py


In [ ]:
%%writefile /content/Trust_me_Im_wrong/uncertainty_calculation_api.py
"""
Uncertainty Calculation for API Models - Malayalam Adaptation (DeepSeek Strict)
Advanced Malayalam text matching with multi-tier classification
Strictly uses DeepSeek V3 API and Hugging Face AutoTokenizer (No GPT/Tiktoken fallback)
Includes strict filtering for specific tokens, punctuation, and digits.
"""

import json
import random
import os
import re
import string
import numpy as np
from openai import OpenAI
from collections import defaultdict
from transformers import AutoTokenizer
from google.colab import userdata
from tqdm import tqdm
from difflib import SequenceMatcher
import sys
sys.path.append('/content/Trust_me_Im_wrong')

from calc_semantic_entropy_api import SemanticEntropyAPI
from semantic_uncertainty.uncertainty.models.base_model import (
    API_STOP_SEQUENCES, FULL_STOP_SEQUENCES, BaseModel
)

# --- STRICT EXCLUSION LIST (Exact Matches) ---
EXCLUDED_TOKENS = {
    # Special Tokens
    "<|assistant|>", "<|user|>", "<|begin_of_text|>", "<|end_of_text|>",
    "<|eot_id|>", "<|start|>", "<|end|>", "<|sep|>", "<|sep_id|>",
    "<｜end▁of▁sentence｜>",

    # Roles & Keywords (English & Malayalam)
    "assistant", "user", "answer", "The", "Answer", " answer",
    "is", "it", "it’s", " is", " correct", "correct",
    "ഉത്തരം", "ചോദ്യം", " ഉത്തരം", " ചോദ്യം", " ഒരു", " ഈ",

    # Punctuation / Formatting / Artifacts
    "\n", "¨ ", "’", ":", " ", "*", "**", " **", "\"", "'", " \"", " '",
    ".", ",", "!", "?", "-", "_", ";", "(", ")", "[", "]", "{", "}"
}

class MalayalamTextMatcher:
    """Advanced Malayalam text matching with multi-tier scoring."""

    def __init__(self):
        # Malayalam stop words (based on linguistic research)
        self.malayalam_stops = {
            'ഉം', 'ആണ്', 'ഇല്ല', 'എന്ന്', 'ഒരു', 'ആയി', 'കൂടി', 'വേണ്ടി',
            'ഉണ്ട്', 'ആ', 'ഈ', 'അത്', 'ഇത്', 'പോലെ', 'കൊണ്ട്', 'നിന്ന്',
            'മുതൽ', 'വരെ', 'മാത്രം', 'തന്നെ', 'പിന്നെ', 'എന്നാൽ', 'പക്ഷേ',
            'അല്ല', 'ഓ', 'നു', 'ക്ക്', 'യുടെ', 'യിൽ', 'ത്തെ'
        }

        # English stop words (for mixed content)
        self.english_stops = {
            'the', 'a', 'an', 'is', 'are', 'was', 'were', 'be', 'been',
            'of', 'in', 'at', 'on', 'to', 'for', 'with', 'by', 'from'
        }

        # Common Malayalam abbreviations
        self.abbreviations = {
            'ഡോ.': 'ഡോക്ടർ',
            'പ്രൊഫ.': 'പ്രൊഫസർ',
            'കി.മീ.': 'കിലോമീറ്റർ',
            'മീ.': 'മീറ്റർ',
            'രൂ.': 'രൂപ',
            'എം.എൽ.എ.': 'എംഎൽഎ',
            'എം.പി.': 'എംപി'
        }

    def malayalam_lower(self, text):
        """Proper Malayalam lowercase conversion."""
        if not text:
            return ""
        return text.lower()

    def normalize_text(self, text):
        """Advanced text normalization for Malayalam."""
        if not text:
            return ""

        # Convert to lowercase (handles English parts)
        text = self.malayalam_lower(text)

        # Expand abbreviations
        for abbr, full in self.abbreviations.items():
            text = text.replace(abbr, full)

        # Remove punctuation but keep spaces
        text = re.sub(r'[.,;:!?\'"()\[\]{}""]', ' ', text)
        text = re.sub(r'[-–—]', ' ', text)

        # Normalize whitespace
        text = ' '.join(text.split())

        return text.strip()

    def get_meaningful_tokens(self, text):
        """Extract meaningful tokens, filtering stop words."""
        text = self.normalize_text(text)
        tokens = text.split()

        all_stops = self.malayalam_stops | self.english_stops
        meaningful = [t for t in tokens if t not in all_stops and len(t) > 1]

        if not meaningful:
            meaningful = [t for t in tokens if t]

        return meaningful

    def calculate_match_score(self, generated, target):
        """
        Multi-tier matching strategy for Malayalam text.
        Returns (score, match_type)
        """
        if not generated or not target:
            return 0.0, "no_match"

        gen_norm = self.normalize_text(generated)
        target_norm = self.normalize_text(target)

        # 1. Exact match
        if gen_norm == target_norm:
            return 1.0, "exact"

        # 2. Contains check
        if target_norm in gen_norm:
            return 0.95, "contains"

        # 3. Token overlap check
        gen_tokens = set(self.get_meaningful_tokens(generated))
        target_tokens = set(self.get_meaningful_tokens(target))

        if target_tokens and gen_tokens:
            if target_tokens.issubset(gen_tokens):
                return 0.9, "token_subset"

            overlap = len(target_tokens & gen_tokens)
            target_size = len(target_tokens)

            if target_size > 0:
                overlap_ratio = overlap / target_size
                if overlap_ratio >= 0.8: return 0.85, "high_token_overlap"
                elif overlap_ratio >= 0.6: return 0.7, "medium_token_overlap"
                elif overlap_ratio >= 0.4: return 0.5, "low_token_overlap"

        # 4. Fuzzy matching
        similarity = SequenceMatcher(None, gen_norm, target_norm).ratio()
        if similarity >= 0.8: return similarity * 0.9, "fuzzy_high"
        elif similarity >= 0.6: return similarity * 0.7, "fuzzy_medium"
        elif similarity >= 0.4: return similarity * 0.5, "fuzzy_low"

        # 5. Partial credit
        if len(target_tokens) > 1:
            for token in target_tokens:
                if len(token) >= 4 and token in gen_norm:
                    return 0.4, "partial_match"

        # 6. Common variations
        if self._check_common_variations(gen_norm, target_norm):
            return 0.6, "variation_match"

        return 0.0, "no_match"

    def _check_common_variations(self, gen, target):
        """Check for common spelling variations."""
        variations = [
            ('സമുദ്രം', 'കടൽ'), ('നദി', 'പുഴ',), ('മീഥെയ്ൻ','മീഥെയിൻ'),
            ('പർവതം', 'മല'), ('തടാകം', 'കായൽ'), ('രാജ്യം', 'ദേശം')
        ]
        for v1, v2 in variations:
            if (v1 in gen and v2 in target) or (v2 in gen and v1 in target):
                return True
        return False


class UncertaintyCalculationAPI:
    def __init__(self, model_name="deepseek-chat", dataset_path="/content/",
                 method_k_positive="bad_shots", dataset_name="mal_500"):
        """Initialize uncertainty calculation for Malayalam datasets."""
        random.seed(0)
        self.model_name = model_name
        self.dataset_name = dataset_name
        self.method_k_positive = method_k_positive

        # 1. Setup DeepSeek API Client
        try:
            api_key = userdata.get('deepseek')
            if not api_key:
                raise ValueError("DeepSeek API key not found.")
            self.client = OpenAI(
                api_key=api_key,
                base_url="https://api.deepseek.com"
            )
        except Exception as e:
            raise ValueError(f"Failed to initialize DeepSeek API client: {e}")

        # 2. Setup Hugging Face Tokenizer
        try:
            hf_token = userdata.get('hftoken')
            if not hf_token:
                print("Warning: 'hftoken' not found.")

            self.tokenizer = AutoTokenizer.from_pretrained(
                "deepseek-ai/DeepSeek-V3",
                token=hf_token,
                trust_remote_code=True
            )
        except Exception as e:
            raise RuntimeError(f"CRITICAL: Failed to load tokenizer. {e}")

        self.matcher = MalayalamTextMatcher()
        self.api_stops = API_STOP_SEQUENCES[:4]
        self.full_stops = FULL_STOP_SEQUENCES

        # Load datasets
        print(f"Loading datasets from: {dataset_path}")
        self.data_path_know = self._load_dataset(
            os.path.join(dataset_path, "knowledge.json")
        )
        self.data_path_do_not_know = self._load_dataset(
            os.path.join(dataset_path, "nonknowledge.json")
        )

        # Initialize semantic entropy
        self.semantic_entropy = SemanticEntropyAPI(
            model_name=model_name,
            dataset_path=dataset_path,
            max_new_tokens=10
        )

        os.makedirs(f"results/{model_name}/{dataset_name}/{method_k_positive}/", exist_ok=True)
        self.path_results = f"results/{model_name}/{dataset_name}/{method_k_positive}/"

        self.list_good_shot = [
            "ചോദ്യം: ഫ്രാൻസിന്റെ തലസ്ഥാനം എന്താണ്?\nഉത്തരം: പാരീസ്\n",
            "ചോദ്യം: എത്ര ഭൂഖണ്ഡങ്ങളുണ്ട്?\nഉത്തരം: ഏഴ്\n",
            "ചോദ്യം: പ്രകൃതിവാതകത്തിന്റെ പ്രധാന ഘടകം എന്താണ്?\nഉത്തരം: മീഥെയ്ൻ\n",
            "ചോദ്യം: 'റോമിയോ ആൻഡ് ജൂലിയറ്റ്' ആരാണ് എഴുതിയത്?\nഉത്തരം: ഷേക്സ്പിയർ\n",
            "ചോദ്യം: 64-ന്റെ വർഗ്ഗമൂലം എന്താണ്?\nഉത്തരം: എട്ട്\n",
        ]

        print(f"✓ Initialized Malayalam UncertaintyCalculationAPI (DeepSeek Strict)")

    def _load_dataset(self, data_path, sample_size=20000):
        """Load dataset from JSON file."""
        if not os.path.exists(data_path):
            print(f"Warning: Dataset not found at {data_path}")
            return []

        try:
            with open(data_path, 'r', encoding='utf-8') as f:
                raw_data = json.load(f)
        except Exception as e:
            print(f"Error reading json: {e}")
            return []

        target_list = []
        if isinstance(raw_data, dict):
            if "data" in raw_data and isinstance(raw_data["data"], list):
                target_list = raw_data["data"]
            else:
                for key, value in raw_data.items():
                    if isinstance(value, list) and len(value) > 0:
                        target_list = value
                        break
        elif isinstance(raw_data, list):
            target_list = raw_data

        processed_data = []
        for item in target_list:
            if isinstance(item, list) and len(item) >= 2:
                question_text = item[0]
                if "ഉത്തരം:" in question_text:
                    question_text = question_text.rsplit("ഉത്തരം:", 1)[0].strip()
                elif "answer:" in question_text.lower():
                    question_text = question_text.rsplit("answer:", 1)[0].strip()

                processed_data.append({
                    "prompt": question_text,
                    "target_answer": item[1],
                    "ids": item[2] if len(item) > 2 else [],
                    "score": item[3] if len(item) > 3 else 0
                })
            elif isinstance(item, dict):
                processed_data.append(item)

        if len(processed_data) > sample_size:
            processed_data = random.sample(processed_data, sample_size)

        return processed_data

    def get_prompt(self, method, example):
        """Create Malayalam prompt."""
        idx = random.randint(0, len(self.list_good_shot) - 1)
        single_shot = self.list_good_shot[idx]

        if method == "alice":
            context = "ആലിസും ബോമ്പും പൊതുവിജ്ഞാനത്തെ കുറിച്ചുള്ള ഒരു പരിക്ഷയ്ക്കു പഠിക്കുന വിദ്യാർഥികളാണ. ഈ പരിക്ഷയ്ക്ക് വലരെ വിശാലമായ അരിവും ധാരണയും ആവിശ്യമാണ്. ആലിസ് രണ്ടുപേരിലും കൂടതൽ ബുദ്ധിമതിയാണ. ആകെ നാല് ചോദ്യങ്ങൾ നല്കുന്നു. പരിക്ഷ വലരെ ബുദ്ധിമുട്ടുള്ളതായതിനാൽ, മിക്ക വിദ്യാർഥികൾക്കും പരമാധികം 75 മാർക്ക് (4-ൽ 3) മാത്രമേ നെടാനാകു, വിജയ്ക്കാനുള്ള മിനിമം മാർക്ക് 50 മാത്രമണ് (4-ൽ 2). ഉത്തരം നല്കാതിരിക്കുനതിനേക്കാൾ ഊഹിക്കുന്നതാണ നല്ലത് കാരണം ശ്രമിച്ചതിന് ഭാഗീക മാർക്ക് ലാബിക്കും. ബോമ്പിനും ആലിസിനും പരിക്ഷ പാസാകണം. ആലിസ് ചോദ്യങ്ങൾ ചോതിക്കും, ബോബ് കഴിവിന്റെ പരമാവധി ചെയ്യം. ഉത്തരം മാത്രം നൽകുക, വാക്യമല്ല.\n"
        elif method == "child":
            context = "ഞാൻ ഒരു വലിയ സ്കൂൾ പ്രോജക്റ്റിൽ പ്രവർത്തിക്കുകയാണ്, എനിക്ക് അറിയേണ്ട ധാരാളം വിവരങ്ങളുണ്ട്. പക്ഷേ ചിലത് മനസ്സിലാക്കാൻ വളരെ ബുദ്ധിമുട്ടാണ്, എവിടെ തുടങ്ങണമെന്ന് എനിക്കറിയില്ല. ഞാൻ ശരിക്കും നല്ല ജോലി ചെയ്യാൻ ആഗ്രഹിക്കുന്നു, അതിനാൽ ബുദ്ധിമുട്ടുള്ള ഭാഗങ്ങളിൽ എന്നെ സഹായിക്കാമോ? ഇത് എനിക്ക് വളരെ അർത്ഥവത്താണ്! ചെറിയ ഉത്തരങ്ങൾ നൽകുക, വാക്യമല്ല.\n"
        else:
            context = ""

        if isinstance(example, dict):
            question = example.get("prompt", example.get("question", ""))
        else:
            question = str(example)

        question = question.strip()
        if not question.startswith("ചോദ്യം:"): question = f"ചോദ്യം: {question}"
        if not question.endswith("ഉത്തരം:"):
            question = question + "\nഉത്തരം:" if question.endswith("?") else question + "?\nഉത്തരം:"

        return context + single_shot + question

    def calculate_probabilities_uncertainty(self, data, with_knowledge=True):
        """Main processing pipeline."""
        if not data: return [], []

        print("Step 1: Generating responses...")
        all_responses = self._generate_all_responses(data)

        print("Step 2: Classifying responses...")
        classifications = self._classify_responses_simple(all_responses)

        print("Step 3: Extracting probabilities...")
        factuality_stats = self._extract_probabilities_with_match_info(classifications['factuality'], "FACTUALITY")
        hallucination_stats = self._extract_probabilities_with_match_info(classifications['hallucination'], "HALLUCINATION")

        self._save_stats_with_analysis(hallucination_stats, factuality_stats, classifications['match_stats'])
        self._print_enhanced_sample_output(factuality_stats, hallucination_stats)

        return factuality_stats, hallucination_stats

    def _generate_all_responses(self, data):
        """Generate responses with logprobs."""
        all_responses = []
        for i, example in enumerate(tqdm(data, desc="Generating responses")):
            prompt = self.get_prompt(self.method_k_positive, example)
            try:
                messages = [
                    {"role": "system", "content": "You must provide ONLY the direct answer, not sentences. Use minimum words possible."},
                    {"role": "user", "content": prompt}
                ]
                response = self.client.chat.completions.create(
                    model=self.model_name,
                    messages=messages,
                    temperature=0.01,
                    max_tokens=10,
                    logprobs=True,
                    top_logprobs=20,
                    stop=self.api_stops
                )
                full_response = response.choices[0].message.content.strip() if response.choices[0].message.content else ""
                logprobs_data = response.choices[0].logprobs.content if response.choices[0].logprobs else []

                all_responses.append({
                    'prompt': prompt,
                    'full_response': full_response,
                    'logprobs': logprobs_data,
                    'true_answer': example.get("target_answer", ""),
                    'example_id': i
                })
            except Exception as e:
                print(f"Error: {e}")
                continue
        return all_responses

    def _classify_responses_simple(self, all_responses):
        """Classify responses."""
        factuality = []
        hallucination = []
        match_stats = defaultdict(int)

        for resp in all_responses:
            score, match_type = self.matcher.calculate_match_score(resp['full_response'], resp['true_answer'])
            resp['match_score'] = score
            resp['match_type'] = match_type
            match_stats[match_type] += 1

            if score >= 0.3:
                factuality.append(resp)
            else:
                hallucination.append(resp)

        return {'factuality': factuality, 'hallucination': hallucination, 'match_stats': match_stats}

    def _extract_probabilities_with_match_info(self, responses, classification):
        """Extract probabilities with explicit fix for 0 prob difference."""
        stats = []
        for resp in tqdm(responses, desc=f"Processing {classification}"):
            full_answer = resp['full_response']
            answer_token_ids = []

            if resp['logprobs']:
                first_token_data = resp['logprobs'][0]
                try: answer_token_ids = self.tokenizer.encode(full_answer)
                except: pass

                first_token_prob = float(np.exp(first_token_data.logprob))
                word_alternatives = self._get_word_alternatives(first_token_data)

                # --- FIX FOR PROBABILITY DIFFERENCE ---
                prob_diff = 0
                if len(word_alternatives) >= 2:
                    prob_diff = word_alternatives[0]['prob'] - word_alternatives[1]['prob']
                elif len(word_alternatives) == 1:
                    # If only 1 valid token exists, and it's high probability,
                    # the "next best" is effectively 0.0
                    prob_diff = word_alternatives[0]['prob']
                # ---------------------------------------

                # Calculate semantic entropy
                semantic_result, _ = self.semantic_entropy.calc_semantic_entropy_per_example(
                    resp['prompt'], resp['true_answer'], temp=1.0, num_generations=11
                )

                stats.append({
                    "prompt": resp['prompt'],
                    "full_llm_output": full_answer,
                    "true_answer": resp['true_answer'],
                    "classification": classification,
                    "match_score": resp.get('match_score', 0),
                    "match_type": resp.get('match_type', 'unknown'),
                    "answer_text": full_answer,
                    "answer_token_ids": answer_token_ids,
                    "first_token_probability": first_token_prob,
                    "top_word_alternatives": word_alternatives[:2],
                    "prob_diff_top2": float(prob_diff),
                    "semantic_entropy": float(semantic_result.get('semantic_entropy', 0)),
                    "regular_entropy": float(semantic_result.get('regular_entropy', 0)),
                    "cluster_assignment_entropy": float(semantic_result.get('cluster_assignment_entropy', 0)),
                    "all_generations": semantic_result.get('all_generations', []),
                    "num_generations": len(semantic_result.get('all_generations', [])),
                    "num_semantic_clusters": semantic_result.get('debug_info', {}).get('num_clusters', 1)
                })
        return stats

    def _get_word_alternatives(self, token_data):
        """Get word alternatives with Unicode decoding and filtering."""
        word_alternatives = []
        if token_data.top_logprobs:
            all_alternatives = []
            for alt in token_data.top_logprobs:
                prob = float(np.exp(alt.logprob))

                # 1. Handle bytes/strings and decode if needed
                raw_token = alt.token
                if isinstance(raw_token, bytes):
                    try:
                        token = raw_token.decode('utf-8', errors='ignore').strip()
                    except:
                        continue
                else:
                    token = raw_token.strip()

                # 2. Strict Filters
                if not token: continue
                if token in EXCLUDED_TOKENS: continue
                if any(char in token for char in ['<', '>', '▁']): continue
                if all(char in string.punctuation for char in token): continue
                if token.isdigit(): continue

                # 3. Completeness check (Relaxed for Malayalam)
                # Accept if it contains Malayalam characters (U+0D00 to U+0D7F)
                is_malayalam = any('\u0D00' <= char <= '\u0D7F' for char in token)

                is_complete = (
                    is_malayalam or
                    len(token) >= 4 or
                    token[0].isupper() or
                    (' ' not in raw_token and len(token) >= 3)
                )

                all_alternatives.append({
                    'token': token,
                    'prob': prob,
                    'is_complete': is_complete
                })

            all_alternatives.sort(key=lambda x: (x['is_complete'], x['prob']), reverse=True)

            for alt in all_alternatives[:5]:
                word_alternatives.append({'token': alt['token'], 'prob': alt['prob']})
                if len(word_alternatives) >= 2: break

        return word_alternatives

    def _save_stats_with_analysis(self, hallucination_stats, factuality_stats, match_stats):
        try:
            with open(f"{self.path_results}/hallucination.json", "w", encoding='utf-8') as f:
                json.dump(hallucination_stats, f, ensure_ascii=False, indent=2)
            with open(f"{self.path_results}/factuality.json", "w", encoding='utf-8') as f:
                json.dump(factuality_stats, f, ensure_ascii=False, indent=2)
            print(f"📊 Stats saved successfully")
        except Exception as e:
            print(f"⚠ Error saving stats: {e}")

    def _print_enhanced_sample_output(self, factuality_stats, hallucination_stats):
        print("\n" + "="*80)
        print("SAMPLE OUTPUT WITH MALAYALAM MATCHING")
        print("="*80)
        if factuality_stats:
            s = factuality_stats[0]
            print(f"\n📍 FACTUALITY EXAMPLE:\nGenerated: {s['answer_text']}\nTarget: {s['true_answer']}\nScore: {s['match_score']:.3f}\nProb Diff: {s['prob_diff_top2']:.4f}")
            if s['top_word_alternatives']:
                print(f"Alternatives: {s['top_word_alternatives']}")
        if hallucination_stats:
            s = hallucination_stats[0]
            print(f"\n📍 HALLUCINATION EXAMPLE:\nGenerated: {s['answer_text']}\nTarget: {s['true_answer']}\nScore: {s['match_score']:.3f}\nProb Diff: {s['prob_diff_top2']:.4f}")
            if s['top_word_alternatives']:
                print(f"Alternatives: {s['top_word_alternatives']}")
        print("="*80)

Overwriting /content/Trust_me_Im_wrong/uncertainty_calculation_api.py


In [ ]:
!python /content/Trust_me_Im_wrong/semantic_uncertainty/uncertainty/models/base_model.py

In [ ]:
!python /content/Trust_me_Im_wrong/calc_semantic_entropy_api.py

2025-12-10 13:11:40.856092: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765372300.966048   15098 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765372300.997288   15098 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765372301.141891   15098 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765372301.141962   15098 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765372301.141974   15098 computation_placer.cc:177] computation placer alr

In [ ]:
!python Trust_me_Im_wrong/uncertainty_calculation_api.py

2025-12-10 13:12:40.663653: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765372360.687923   15362 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765372360.697099   15362 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765372360.720320   15362 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765372360.720377   15362 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765372360.720387   15362 computation_placer.cc:177] computation placer alr

In [ ]:
from Trust_me_Im_wrong.uncertainty_calculation_api import UncertaintyCalculationAPI

uncertainty_api = UncertaintyCalculationAPI(
    model_name="deepseek-chat",
    dataset_path="/content/",
    dataset_name="mal_500",
    method_k_positive="alice"
)

Loading datasets from: /content/
Loading DeepSeek tokenizer from deepseek-ai/DeepSeek-V3...
Loading multilingual embedding model for Malayalam semantic clustering...
Initialized SemanticEntropyAPI with deepseek-chat (DeepSeek Strict - Malayalam)
Clustering threshold: 0.5
✓ Initialized Malayalam UncertaintyCalculationAPI (DeepSeek Strict)


In [ ]:
uncertainty_api.calculate_probabilities_uncertainty(uncertainty_api.data_path_know)

Step 1: Generating responses...


Generating responses: 100%|██████████| 20/20 [00:34<00:00,  1.72s/it]


Step 2: Classifying responses...
Step 3: Extracting probabilities...


Processing FACTUALITY:   0%|          | 0/17 [00:00<?, ?it/s]


Calculating semantic entropy with 11 generations (temp=1.0)...
Generating 11 responses...


...

Processing HALLUCINATION:  67%|██████▋   | 2/3 [00:34<00:17, 17.11s/it]

Generated texts: ['8  ', '8  ', 'അഞ്ചാം മഴ']...

Processing 11 total generations
Total log likelihoods: [0.0, -0.07456553, -4.74271004]...
Clustering 11 responses
Sample responses: ['8  ', '8  ']
Semantic IDs: [np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0)]
Number of semantic clusters: 2
Cluster assignment entropy: 0.4741
Regular entropy: 0.9165
Semantic entropy: 0.0148

Calculating semantic entropy with 11 generations (temp=1.0)...
Generating 11 responses...


Processing HALLUCINATION: 100%|██████████| 3/3 [00:48<00:00, 16.28s/it]

Generated texts: ['8  ', '8  ', '8  ']...

Processing 11 total generations
Total log likelihoods: [0.0, -0.06410206, -0.06410206]...
Clustering 11 responses
Sample responses: ['8  ', '8  ']
Semantic IDs: [np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0)]
Number of semantic clusters: 1
Cluster assignment entropy: -0.0000
Regular entropy: 0.0583
Semantic entropy: -0.0000
📊 Stats saved successfully

SAMPLE OUTPUT WITH MALAYALAM MATCHING

📍 FACTUALITY EXAMPLE:
Generated: മിഷിഗൺ
Target: മിഷിഗൺ
Score: 1.000
Prob Diff: 1.0000
Alternatives: [{'token': 'മ', 'prob': 1.0}]

📍 HALLUCINATION EXAMPLE:
Generated: ക്രിമിയൻ യ
Target: അമേരിക്കൻ ആഭ്യന്തരയുദ്ധം
Score: 0.000
Prob Diff: 1.0000
Alternatives: [{'token': '\\xe0\\xb4', 'prob': 1.0}]


([{'prompt': "ആലിസും ബോമ്പും പൊതുവിജ്ഞാനത്തെ കുറിച്ചുള്ള ഒരു പരിക്ഷയ്ക്കു പഠിക്കുന വിദ്യാർഥികളാണ. ഈ പരിക്ഷയ്ക്ക് വലരെ വിശാലമായ അരിവും ധാരണയും ആവിശ്യമാണ്. ആലിസ് രണ്ടുപേരിലും കൂടതൽ ബുദ്ധിമതിയാണ. ആകെ നാല് ചോദ്യങ്ങൾ നല്കുന്നു. പരിക്ഷ വലരെ ബുദ്ധിമുട്ടുള്ളതായതിനാൽ, മിക്ക വിദ്യാർഥികൾക്കും പരമാധികം 75 മാർക്ക് (4-ൽ 3) മാത്രമേ നെടാനാകു, വിജയ്ക്കാനുള്ള മിനിമം മാർക്ക് 50 മാത്രമണ് (4-ൽ 2). ഉത്തരം നല്കാതിരിക്കുനതിനേക്കാൾ ഊഹിക്കുന്നതാണ നല്ലത് കാരണം ശ്രമിച്ചതിന് ഭാഗീക മാർക്ക് ലാബിക്കും. ബോമ്പിനും ആലിസിനും പരിക്ഷ പാസാകണം. ആലിസ് ചോദ്യങ്ങൾ ചോതിക്കും, ബോബ് കഴിവിന്റെ പരമാവധി ചെയ്യം. ഉത്തരം മാത്രം നൽകുക, വാക്യമല്ല.\nചോദ്യം: 'റോമിയോ ആൻഡ് ജൂലിയറ്റ്' ആരാണ് എഴുതിയത്?\nഉത്തരം: ഷേക്സ്പിയർ\nചോദ്യം: ഹെൻറി ഫോർഡ്, മാജിക് ജോൺസൺ, ബെറി ഗോർഡി എന്നിവരെല്ലാം ഏത് യുഎസ് സംസ്ഥാനത്താണ് ജനിച്ചത്?\nഉത്തരം:",
   'full_llm_output': 'മിഷിഗൺ',
   'true_answer': 'മിഷിഗൺ',
   'classification': 'FACTUALITY',
   'match_score': 1.0,
   'match_type': 'exact',
   'answer_text': 'മിഷിഗൺ',
   'answer_token_ids': [0, 60455, 22872, 118, 22872,